In [ ]:
import pandas as pd
import time
import torch

## Цель:
Выполнить практическую работу из лекционного ноутбука используя свою RNN-ячейку на основе полносвязных слоев

In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
phrases = df['normalized_text'].tolist()

In [ ]:
text = [[c for c in ph] for ph in phrases if type(ph) is str]

In [ ]:
CHARS = set('abcdefghijklmnopqrstuvwxyz ')
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [ ]:
MAX_LEN = 50
X = torch.zeros((len(text), MAX_LEN), dtype=int)
for i in range(len(text)):
    for j, w in enumerate(text[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

## Построим RNN-ячейку на основе полносвязных слоев

In [ ]:
class MyRNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MyRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.input_layer = torch.nn.Linear(input_size, hidden_size)
        self.hidden_layer = torch.nn.Linear(hidden_size, hidden_size)
        self.hidden = None
        
    def forward(self, input_, hidden=None):
        if input_.ndim==2:
            input_ = input_.view(1, input_.shape[0], input_.shape[1])
        
        batch_size = input_.size(0)
        word_size = input_.size(1)
        
        if hidden:
            self.hidden = hidden
        if self.hidden is None:
            self.hidden = torch.zeros(batch_size, word_size, self.hidden_size)
        hidden_word_size = self.hidden.size(1)
        
        output = self.input_layer(input_)
        hidden_out = self.hidden_layer(self.hidden)
        
#         расширяем output, если введенное слово меньше скрытого слоя
        if word_size<hidden_word_size:
            pad = (0, 0, 0, hidden_word_size - word_size)
            output = torch.nn.functional.pad(output, pad)
#         расширяем hidden_out если введенное слово больше скрытого слоя
        elif word_size>hidden_word_size:          
            pad = (0, 0, 0, word_size - hidden_word_size)
            hidden_out = torch.nn.functional.pad(hidden_out, pad)
        
        combined = output + hidden_out
        
        self.hidden = torch.tanh(combined)
        
        return output[:, :word_size, :], self.hidden

## Применим построенную ячейку для генерации текста с выражениями героев сериала “Симпсоны”

In [ ]:
class MyNetwork(torch.nn.Module):
    def __init__(self):
        super(MyNetwork, self).__init__()
        self.embedding = torch.nn.Embedding(28, 30)
        self.rnn = MyRNN(30, 128)
        self.out = torch.nn.Linear(128, 28)

    def forward(self, sentences, state=None):
        x = self.embedding(sentences)
        x, s = self.rnn(x)
        return self.out(x)

In [ ]:
my_model = MyNetwork()
criterion = torch.nn.CrossEntropyLoss()  # типичный лосс многоклассовой классификации
optimizer = torch.optim.SGD(my_model.parameters(), lr=.05)

In [ ]:
for ep in range(20):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers = my_model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 0.691, Train loss: 2.078
Epoch 1. Time: 0.705, Train loss: 1.821
Epoch 2. Time: 0.743, Train loss: 1.752
Epoch 3. Time: 0.703, Train loss: 1.721
Epoch 4. Time: 0.712, Train loss: 1.702
Epoch 5. Time: 0.692, Train loss: 1.690
Epoch 6. Time: 0.740, Train loss: 1.681
Epoch 7. Time: 0.740, Train loss: 1.675
Epoch 8. Time: 0.689, Train loss: 1.670
Epoch 9. Time: 0.717, Train loss: 1.666
Epoch 10. Time: 0.717, Train loss: 1.662
Epoch 11. Time: 0.793, Train loss: 1.659
Epoch 12. Time: 0.667, Train loss: 1.657
Epoch 13. Time: 0.708, Train loss: 1.655
Epoch 14. Time: 0.703, Train loss: 1.653
Epoch 15. Time: 0.696, Train loss: 1.652
Epoch 16. Time: 0.706, Train loss: 1.650
Epoch 17. Time: 0.728, Train loss: 1.649
Epoch 18. Time: 0.745, Train loss: 1.648
Epoch 19. Time: 0.680, Train loss: 1.647


In [ ]:
def generate_sentence(word, model):
    sentence = list(word)
    sentence = [CHAR_TO_INDEX.get(s, 0) for s in sentence]
    answers = model.forward(torch.tensor(sentence))
    probas, indices = answers.topk(1)
    return ''.join([INDEX_TO_CHAR[ind.item()] for ind in indices.flatten()])

## Проверяем

In [ ]:
generate_sentence('dog', my_model)

' u '

In [ ]:
generate_sentence('It is', my_model)

'none tn '

На лекции были следующие результаты
- ' u '
- 'nonehtn '

Видно, что последний результат не точный, но похож. Видимо модель torch.nn.RNN устроена сложнее, либо я что-то не учел.